# Лабораторная работа № 8. Определение критически важных элементов модельной РЛС.

В работе предлагается провести свободный анализ функционирования модельной РЛС и выявить критически важные элементы, влияющие больше всего на излучаемую мощность РЛС.

## Описание данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('Lab8_data.csv')
df.head(10)

,Timestamp,Regime,Power1,Power2,Power3,Power4,Power5,Power6,Power7,Power8,...,State10,State11,State12,State13,State14,State15,State16,Sum_power,State_float,State
0,0,1,261.39,340.49,334.45,340.37,344.02,0.00,302.29,0.00,...,0,0,0,1,0,1,1,3051.94,0.55,0
1,1,1,0.00,340.99,339.69,90.64,344.48,0.00,306.04,0.00,...,0,0,0,1,0,1,1,2553.82,0.49,0
2,2,1,0.00,335.54,340.03,339.24,344.12,0.00,302.10,0.00,...,0,0,0,1,0,1,1,2794.18,0.55,0
3,3,1,0.00,334.77,340.07,73.36,350.73,0.00,303.01,0.00,...,0,0,0,1,0,1,1,2528.75,0.48,0
4,4,2,0.00,346.08,316.63,334.01,351.67,0.00,0.00,0.00,...,0,0,0,1,0,1,0,2242.65,0.43,0
5,5,3,0.00,371.02,334.63,340.65,312.12,431.53,0.00,387.62,...,1,1,1,1,1,1,1,5260.90,0.80,0
6,6,1,0.00,334.05,338.59,334.71,347.14,402.03,0.00,398.57,...,1,1,1,1,1,1,1,5295.45,0.89,0
7,7,1,0.00,334.94,337.05,337.98,350.29,401.48,0.00,397.23,...,1,1,1,1,1,0,1,5046.38,0.82,0
8,8,2,0.00,338.99,318.45,176.96,365.41,373.61,0.00,405.66,...,1,1,1,1,0,1,1,4570.55,0.73,0
9,9,1,0.00,334.78,338.79,339.89,345.30,403.50,0.00,401.84,...,1,1,1,1,1,1,1,5288.24,0.88,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 37 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Timestamp    100000 non-null  int64  
 1   Regime       100000 non-null  int64  
 2   Power1       100000 non-null  float64
 3   Power2       100000 non-null  float64
 4   Power3       100000 non-null  float64
 5   Power4       100000 non-null  float64
 6   Power5       100000 non-null  float64
 7   Power6       100000 non-null  float64
 8   Power7       100000 non-null  float64
 9   Power8       100000 non-null  float64
 10  Power9       100000 non-null  float64
 11  Power10      100000 non-null  float64
 12  Power11      100000 non-null  float64
 13  Power12      100000 non-null  float64
 14  Power13      100000 non-null  float64
 15  Power14      100000 non-null  float64
 16  Power15      100000 non-null  float64
 17  Power16      100000 non-null  float64
 18  State1       100000 non-n

Файл с данными предствляет собой запись мощности, излучаемой модельной РЛС, состоящей из 16 блоков усиления мощности (БУМ). Динамика мощности отражает реальные процессы и взаимосвязи, происходящие на РЛС.

Первый столбец обозначает временной отсчет, второй столбец - режим работы РЛС, который напрямую определяет выходную мощность БУМ и сказывается на внутренних процессах деградации БУМ. Далее в 16 столбцах записаны мощности выходного сигнала БУМ, а также состояния каждого из блоков усиления мощности, которое показывает, дает ли этот блок вклад в излучаемую мощность РЛС. Следом представлена суммарная мощность, излучаемая РЛС, а также состояние РЛС в виде вещественного числа, которое показывает, насколько излучаемая мощность соответствует максимально возможной при данном режиме работы РЛС. Последний столбец - информация о состоянии РЛС в виде бинарной величины.

Каждый БУМ характеризуется своим состоянием, который показывает, насколько выдаваемая мощность соответствует максимально возможной для этого БУМ. Снижение работоспособности БУМ (снижение его состояния) связано исключительно с работой самого БУМ: чем дольше и напряженнее он работает, тем быстрее снижается его состояние. Кроме того, блоки немного отличаются друг от друга, т.е. каждый из них снижает свою рабоспособность немного с разной скоростью.

В каждом из режимов работы есть диапазон возможных излучаемых мощностей. В самом простом режиме работы (зондирование местности) изменение состояния БУМ не происходит. В самом напряженном режиме работы (сопровождение одновременно нескольких целей) уменьшение состояния БУМ происходит быстрее всего.

При снижении выдаваемой мощности ниже определенной отметки, оператор принимает решение о замене текущего БУМ новым, и блок перестает работать - его выходная мощность равна 0. Если таких блоков накопится несколько, то они заменяеются на абсолютно новые, состояние которых равно 1.

Иногда мощность блоков независимо от действующего режима опускается ниже 300 Вт (случайное резкое снижение мощности) на 1 отсчет. После этого блоки продолжают работать также, как и ранее.

Каждый БУМ имеет свой вклад в суммарную мощность РЛС. Суммарная излучаемая мощность РЛС равна сумме выдаваемой мощности каждого БУМ, умноженного на коэффициент влияния БУМ (вещественное число от 0 до 1). Под отказом РЛС (значение 0 в последнем столбце) понимается снижение суммарной мощности на 10 % от максимально возможной при заданном режиме работы.

## Задания

### 1. Определите диапазон выходных мощностей БУМ для каждого из режимов работы. Определите среднюю мощность и стандартное отклонение. Изобразите эту информацию визуально.

### 2. Определите для каждого БУМ его подверженность деградации при работе по назначению. Изобразите эту информацию визуально.

### 3. Определите вклад каждого БУМ в излучаемую мощность РЛС. Изобразите эту информацию визуально. Выделите критически важные элементы - блоки, которые в большей степени влияют на излучаемую мощность РЛС.

### 4. Определите, каково среднее время жизни каждого БУМ. Изобразите эту информацию визуально.

### 5. Сделайте выводы.